In [1]:
# Vanilla RNN from scratch using NumPy
# - character-level next-step prediction example helpers included
# - forward, loss, backward (BPTT), parameter update (SGD with grad clipping)
# Paste this whole cell into Jupyter.

import numpy as np

class VanillaRNN:
    def __init__(self, input_size, hidden_size, output_size, seq_len=25, learning_rate=1e-1, clip=5.0, seed=42):
        """
        input_size : size of one-hot input (vocab size for char-RNN)
        hidden_size: number of hidden units
        output_size: size of output (same as input_size for next-char prediction)
        seq_len    : truncated BPTT length
        learning_rate: SGD step size
        clip       : gradient clipping threshold (L2)
        """
        np.random.seed(seed)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.seq_len = seq_len
        self.lr = learning_rate
        self.clip = clip

        # Initialize weights (small random numbers)
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01   # input -> hidden
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden -> hidden
        self.Why = np.random.randn(output_size, hidden_size) * 0.01  # hidden -> output
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

        # For simple SGD we don't need momentum or RMSProp here; you can add later.

    @staticmethod
    def softmax(x):
        e = np.exp(x - np.max(x, axis=0, keepdims=True))
        return e / np.sum(e, axis=0, keepdims=True)

    def forward(self, inputs, hprev):
        """
        inputs: list of integers (one-hot indices) length T
        hprev: initial hidden column vector shape (hidden_size, 1)
        Returns: loss, activations, last hidden
        """
        xs, hs, ys, ps = {}, {}, {}, {}
        hs[-1] = np.copy(hprev)
        loss = 0
        # forward pass
        for t, ix in enumerate(inputs):
            x = np.zeros((self.input_size, 1))
            x[ix] = 1
            xs[t] = x
            hs[t] = np.tanh(self.Wxh.dot(x) + self.Whh.dot(hs[t-1]) + self.bh)
            ys[t] = self.Why.dot(hs[t]) + self.by
            ps[t] = self.softmax(ys[t])
        return xs, hs, ys, ps

    def compute_loss(self, ps, targets):
        """
        ps: predicted probs dict t->(vocab,1)
        targets: list of integer targets, same length as ps
        returns: scalar cross-entropy loss
        """
        loss = 0
        for t in range(len(targets)):
            loss += -np.log(ps[t][targets[t], 0] + 1e-12)
        return loss

    def backward(self, xs, hs, ps, targets):
        """
        Backprop through time (truncated to len(targets))
        Returns gradients for all parameters and last hidden state gradient
        """
        # initialize gradients
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)
        dhnext = np.zeros((self.hidden_size, 1))

        T = len(targets)
        for t in reversed(range(T)):
            dy = np.copy(ps[t])
            dy[targets[t]] -= 1  # softmax derivative (cross-entropy)
            dWhy += dy.dot(hs[t].T)
            dby += dy

            dh = self.Why.T.dot(dy) + dhnext  # backprop into h
            # backprop through tanh nonlinearity
            dhraw = (1 - hs[t] * hs[t]) * dh
            dbh += dhraw
            dWxh += dhraw.dot(xs[t].T)
            dWhh += dhraw.dot(hs[t-1].T)
            dhnext = self.Whh.T.dot(dhraw)

        # clip gradients to avoid exploding gradients
        for grad in (dWxh, dWhh, dWhy, dbh, dby):
            np.clip(grad, -self.clip, self.clip, out=grad)

        return {'Wxh': dWxh, 'Whh': dWhh, 'Why': dWhy, 'bh': dbh, 'by': dby}, hs[T-1]

    def update_params(self, grads):
        # Simple SGD update
        self.Wxh -= self.lr * grads['Wxh']
        self.Whh -= self.lr * grads['Whh']
        self.Why -= self.lr * grads['Why']
        self.bh  -= self.lr * grads['bh']
        self.by  -= self.lr * grads['by']

    def sample(self, seed_ix, n, h=None):
        """
        Sample a sequence of length n, given a seed index seed_ix and start hidden state h.
        Returns list of sampled indices.
        """
        if h is None:
            h = np.zeros((self.hidden_size, 1))
        x = np.zeros((self.input_size, 1))
        x[seed_ix] = 1
        ixes = []
        for t in range(n):
            h = np.tanh(self.Wxh.dot(x) + self.Whh.dot(h) + self.bh)
            y = self.Why.dot(h) + self.by
            p = self.softmax(y)
            # sample from p
            ix = np.random.choice(range(self.input_size), p=p.ravel())
            x = np.zeros((self.input_size, 1))
            x[ix] = 1
            ixes.append(ix)
        return ixes


In [2]:
# Training example: learn to predict next character for a very small corpus
# Paste as a new cell below the previous one and run both.

data = "hello world\n" * 200  # small corpus repeated to give enough steps
chars = sorted(list(set(data)))
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for ch, i in char_to_ix.items()}

print("Vocab size:", vocab_size)
print("Chars:", chars)

# Hyperparameters
hidden_size = 100
seq_len = 25
learning_rate = 1e-1
iterations = 3000
print_every = 300

# Initialize model
rnn = VanillaRNN(input_size=vocab_size, hidden_size=hidden_size, output_size=vocab_size,
                 seq_len=seq_len, learning_rate=learning_rate)

# Prepare data as list of indices
data_ix = [char_to_ix[ch] for ch in data]
data_len = len(data_ix)

# Training loop (simple SGD over sequence chunks)
p = 0  # data pointer
smooth_loss = -np.log(1.0 / vocab_size) * seq_len

for it in range(1, iterations + 1):
    # prepare inputs and targets
    if p + seq_len + 1 >= data_len or it == 1:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0  # go from start of data
    inputs = data_ix[p:p + seq_len]
    targets = data_ix[p + 1:p + seq_len + 1]

    # forward
    xs, hs, ys, ps = rnn.forward(inputs, hprev)
    loss = rnn.compute_loss(ps, targets)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # backward
    grads, hprev = rnn.backward(xs, hs, ps, targets)
    rnn.update_params(grads)

    p += seq_len  # move data pointer

    if it % print_every == 0 or it == 1:
        print(f"Iter {it}, loss: {smooth_loss:.4f}")
        # sample from the model
        sample_ix = rnn.sample(seed_ix=inputs[0], n=50, h=hs[seq_len-1])
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print("----\n", txt, "\n----")

# Final sample after training
seed = char_to_ix['h']
sample_ix = rnn.sample(seed_ix=seed, n=100)
print("Final sample:\n", ''.join(ix_to_char[ix] for ix in sample_ix))


Vocab size: 9
Chars: ['\n', ' ', 'd', 'e', 'h', 'l', 'o', 'r', 'w']
Iter 1, loss: 54.9306
----
 whd 
lldoodeweohoolrwldeol 
odhol
 ddlh 
 whrrho o 
----
Iter 300, loss: 130.3570
----
 o dlo dlo dlo dlo dlo dlo dlo dlo dlo dlo dlo dlo  
----
Iter 600, loss: 186.6912
----
 
o w
o w
o w
o w
o w
o w
o w
o w
o w
o w
o w
o w
o 
----
Iter 900, loss: 227.1397
----
 lloelloelloelloelloelloelloelloelloelloelloelloell 
----
Iter 1200, loss: 258.6012
----
 elloelloelloelloelloelloelloelloelloelloelloelloel 
----
Iter 1500, loss: 281.1189
----
 rellrel
rel
rel
rel
rellrel
rel
rellrel
rel
rel
re 
----
Iter 1800, loss: 298.4631
----
 
h d
h d
h dlh d
h d
h d
h d
h d
h d
h dlh d
h d
h 
----
Iter 2100, loss: 310.1957
----
 orlworlworlworlworlworlworlworlworlworlworlworlwor 
----
Iter 2400, loss: 319.8803
----
 wor wor wor wor wor wor wor wor wor wor wor wor wo 
----
Iter 2700, loss: 328.0183
----
  dor dor dor dor dor dor dor dor dor dor dor dor d 
----
Iter 3000, loss: 331.4525
----
 r w
r w
r w
r w
r